## 01. 建立負樣本的標註檔

In [ ]:
fp = open('Haar-Training-master/Haar-Training-master/training/negative/bg.txt','w') 
files = glob.glob('Haar-Training-master/Haar-Training-master/training/negative/*.bmp') 
text = "" 

for file in files: 
    basename = os.path.basename(file) 
    filename = 'negative/'+ basename 
    text += filename + "\n" 

print(text) 
fp.write(text) 
fp.close()

## 02. 調整正樣本的影像與標註檔案

In [ ]:
fp = open('Haar-Training-master/Haar-Training-master/training/positive/info.txt', 'r') 
lines = fp.readlines()
rettext = '' 

for line in lines: 
    data = line.split(' ') 
    rettext += data[0] + ' ' + data[1] + ' ' 
    n = data[1]
    for i in range(int(n)): 
        x = float(data[2+i*4]) 
        y = float(data[3+i*4]) 
        w = float(data[4+i*4]) 
        h = float(data[5+i*4]) 
        if (w/h) < 3.8: 
            newW = h * 3.8 
            x -= int((newW - w) / 2) 
                if x<=0: 
                    x=0 
            w = int(newW) 
        rettext = rettext+str(int(x))+' '+data[3+i*4]+' '+str(int(w))+' '+data[5+i*4] 
fp.close() 

fp = open('Haar-Training-master/Haar-Training-master/training/positive/info.txt', 'w') 
fp.write(rettext) 
fp.close()

## 影像增量

In [ ]:
path = 'Haar-Training-master/Haar-Training-master/training/positive/' 
fp = open(path + 'info.txt', 'r') 
lines = fp.readlines() 
count = len(glob.glob("carPlate/*.bmp")) 

rettext = '' 
for line in lines: 
data = line.split(' ') 
    img = Image.open(path + data[0]) 

    x = int(data[2]) 
    y = int(data[3]) 
    w = int(data[4]) 
    h = int(data[5]) 

    reduceW = 30 #減少的的寬度 
    reduceH = int(reduceW*0.75) #減少的的高度 
    multi = float(300/(300-reduceW)) #原圖與新圖比例 
    neww = int(w*multi) #新圖的寬 
    newh = int(h*multi) #新圖的高 

    #移除左上角圖 
    if (x-reduceW)>5 and (y-reduceH)>5: #左上角有空間才移除左上角 
        count += 1 #編號加1,此數值會做為檔名用 
        newimg = img.crop((reduceW, reduceH, 300, 225)) #擷取圖形 
        newimg = newimg.resize((300, 225), Image.ANTIALIAS) #放大圖形 
        newimg.save(path + 'rawdata/bmpraw{:0>3d}.bmp'.format(count), 'bmp') #存檔 
        newx = int((x-reduceW)*multi) #新圖X坐標 
        newy = int((y-reduceH)*multi) #新圖Y坐標 
        rettext = rettext+'rawdata/bmpraw{:0>3d}.bmp'.format(count)+' '+'1'+' '+str(newx)+' '+str(newy)+' '+str(neww)+' '+str(newh)+'\n' #記錄新影像資料

    #移除右上角圖
    if (x+w)<(300-reduceW-5) and y>(reduceW+5):
        count += 1
        newimg = img.crop((0, reduceH, (300-reduceW), 225))
        newimg = newimg.resize((300, 225), Image.ANTIALIAS)
        newimg.save(path + 'rawdata/bmpraw{:0>3d}.bmp'.format(count), 'bmp')
        newx = int(x*multi)
        newy = int((y-reduceH)*multi)
        rettext = rettext+'rawdata/bmpraw{:0>3d}.bmp'.format(count)+' '+'1'+' '+str(newx)+' '+str(newy)+' '+str(neww)+' '+str(newh)+'\n'

    #移除左下角圖
    if (x-reduceW)>5 and (y+h)<(225-reduceH-5):
        count += 1
        newimg = img.crop((reduceW, 0, 300, 225-reduceH))
        newimg = newimg.resize((300, 225), Image.ANTIALIAS)
        newimg.save(path + 'rawdata/bmpraw{:0>3d}.bmp'.format(count), 'bmp')
        newx = int((x-reduceW)*multi)
        newy = int(y*multi)
        rettext = rettext+'rawdata/bmpraw{:0>3d}.bmp'.format(count)+' '+'1'+' '+str(newx)+' '+str(newy)+' '+str(neww)+' '+str(newh)+'\n'

    #移除右下角圖
    if (x+w)<(300-reduceW-5) and (y+h)<(225-reduceH-5):
        count += 1
        newimg = img.crop((0, 0, (300-reduceW), 225-reduceH))
        newimg = newimg.resize((300, 225), Image.ANTIALIAS)
        newimg.save(path + 'rawdata/bmpraw{:0>3d}.bmp'.format(count), 'bmp')
        newx = int(x*multi)
        newy = int(y*multi)
        rettext = rettext+'rawdata/bmpraw{:0>3d}.bmp'.format(count)+' '+'1'+' '+str(newx)+' '+str(newy)+' '+str(neww)+' '+str(newh)+'\n'

fp.close()
    
fpmake = open(path + 'Info.txt', 'a')  #以新增資料方式開啟檔案
fpmake.write(rettext)  #寫入檔案
fpmake.close()